<a href="https://colab.research.google.com/github/Olivia-Feldman/NUGAN-DISTGAN/blob/Olivia/DIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np




In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [4]:
print(train_dataset.data.shape)

torch.Size([60000, 28, 28])


In [6]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
      
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(x.size(0),-1)
        return x

In [7]:
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)

     # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [8]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))

In [9]:
lr = 0.0002 
z_dim = 100
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)
print(mnist_dim)


G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

784


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [10]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

In [11]:
encoder = autoencoder()
#discriminator = Discriminator()
#generator = Generator()



In [12]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
discriminator = Discriminator(784)

In [13]:
adversarial_loss = torch.nn.BCELoss()

In [44]:
num_epochs = 30
for epoch in range(num_epoches):
    for  data in train_loader:
        imgs, _ = data
        imgs = imgs.view(imgs.size(0), -1)
        print(imgs.shape)
       
  
  
        # Adversarial ground truths
        valid = Variable(Tensor( imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure real images 
        real_imgs = Variable(imgs).type(Tensor)
      

        ## Train Generator##
        
        G_optimizer.zero_grad()

      # encoder to create encoded imgs for generator 
        encoded_imgs = encoder(real_imgs)
        print(encoded_imgs.shape)
        print(real_imgs.shape)
      

        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * torch.nn.L1Loss()(
            encoded_imgs, real_imgs
        )

        g_loss.backward()
        G_optimizer.step()

       
        #  Train Discriminator ###
     

        D_optimizer.zero_grad()

        # Sample noise as discriminator ground truth
        z = Variable(transforms.Tensor(np.random.normal(0, 1, (imgs.shape[0], 784))))

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        D_optimizer.step()

        print(
            "[Epoch %d/%d], [D loss: %f] ,[G loss: %f]"%(epoch,num_epochs, d_loss.item(), g_loss.item())
        )

       
        

torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.691169] ,[G loss: 0.943578]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.693270] ,[G loss: 0.943471]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.693224] ,[G loss: 0.943244]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.692023] ,[G loss: 0.943841]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.693206] ,[G loss: 0.943993]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.692469] ,[G loss: 0.944149]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.696918] ,[G loss: 0.945103]
torch.Size([100, 784])
torch.Size([100, 784])
torch.Size([100, 784])
[Epoch 0/30], [D loss: 0.693824] ,[G loss: 0.947031]
torch.Size([100, 784])
t

KeyboardInterrupt: ignored